# Convert a picture dataset using ESIM - part 1/2

This serie of notebooks is an example of how to convert a picture dataset using RPG ESIM (available: https://github.com/uzh-rpg/rpg_esim), a ROS-based event camera simulator developed by the Robotic Perception Group (UZH).

The choosen dataset is "IBERBIRDS: A dataset of flying bird species present in the Iberian Peninsula" (available: https://zenodo.org/records/15090760), whose pictures were reorganized in order to have a single folder for every specie. There is a total of 4000 pictures divided in 10 folders (one for each specie). 

*Please select Kernel → Python 2 before proceeding.*

In the following part, the BAG files obtained in this notebook will be converted in AEDAT files.

In [1]:
import os
import cv2
import time
import shutil
import subprocess
import numpy as np

## Create the frames 

The script below creates the frames for all images using some functions. The first one reads the YOLO annotations, the second one crops the image to a square and the third one applies a zoom factor to it to create the frames. Moreover, a CSV file with timestamps for each frame is created.

In [2]:
def read_yolo(label_path, img_w, img_h):
    with open(label_path, 'r') as f:
        line = f.readline()
        parts = line.strip().split()
        x, y, w, h = map(float, parts[1:]) # first value in the file not needed
        x_c, y_c = x * img_w, y * img_h
        bw, bh = w * img_w, h * img_h
        x1, y1 = int(x_c - bw/2), int(y_c - bh/2)
        x2, y2 = int(x_c + bw/2), int(y_c + bh/2)
        return x1, y1, x2, y2
    
def crop_square(img, bbox, out_size):
    h, w = img.shape[:2]
    x1, y1, x2, y2 = bbox
    cx, cy = (x1 + x2)//2, (y1 + y2)//2
    bbox_size = max(x2 - x1, y2 - y1)
    half = max(bbox_size, min(h, w)) // 2
    left = max(0, cx - half)
    right = min(w, cx + half)
    top = max(0, cy - half)
    bottom = min(h, cy + half)
    cropped = img[top:bottom, left:right]
    cropped = cv2.resize(cropped, (out_size, out_size), interpolation=cv2.INTER_LINEAR)
    return cropped

def zoom_image(img, zoom_factor):
    h, w = img.shape[:2]
    nh, nw = int(h / zoom_factor), int(w / zoom_factor)
    y1 = (h - nh) // 2
    x1 = (w - nw) // 2
    cropped = img[y1:y1+nh, x1:x1+nw]
    cropped = cv2.resize(cropped, (w, h), interpolation=cv2.INTER_LINEAR)
    return cropped

In [ ]:
image_root = os.path.expanduser("~/Images") 
label_root = os.path.expanduser("~/Labels")
sequence_root = os.path.expanduser("~/Sequences") 

num_frames = 100
zoom_min = 1.0
zoom_max = 5.0
square_size = 512

if not os.path.isdir(sequence_root):
    os.makedirs(sequence_root)

for specie in sorted(os.listdir(image_root)):
    specie_path = os.path.join(image_root, specie)
    if not os.path.isdir(specie_path):
        continue
        
    for image_name in sorted(os.listdir(specie_path)):
        if not image_name.lower().endswith(".png"):
            continue

        image_path = os.path.join(specie_path, image_name)
        img = cv2.imread(image_path)
        if img is None:
            continue

        base_name = os.path.splitext(image_name)[0]
        
        sequence_path = os.path.join(sequence_root, specie, "{}_{}".format(specie, base_name))
        if os.path.exists(sequence_path):
            print("\nSkipping already processed image: {}".format(sequence_path))
            continue
            
        frames_path = os.path.join(sequence_path, "cam0", "frames")
        if not os.path.exists(frames_path):
            os.makedirs(frames_path)
            print("\nCreating the new folder: {}".format(frames_path))
        
        label_path = os.path.join(label_root, specie, "{}.{}".format(base_name, "txt"))
        print("Checking label path: {}".format(label_path))
        if not os.path.exists(label_path):
            print("Label file not found for image: {}".format(image_path))
            continue
            
        h, w = img.shape[:2]
        bbox = read_yolo(label_path, w, h)
        cropped_img = crop_square(img, bbox, square_size)
        
        for i in range(num_frames):
            zoom = zoom_min + (zoom_max - zoom_min) * i / (num_frames - 1)
            frame = zoom_image(cropped_img, zoom)
            frame_name = "frame_{}.png".format(str(i).zfill(3))
            cv2.imwrite(os.path.join(frames_path, frame_name), frame)
            
        cmd = "python /home/asia/sim_ws/src/rpg_esim/event_camera_simulator/esim_ros/scripts/generate_stamps_file.py -i {} -r 50.0".format(os.path.abspath(frames_path))
        subprocess.call(cmd, shell=True)

        print("The following image was correctly zoomed: {}".format(image_path))

## Obtain the events

The following functions are used to kill existing roscores (if any), start a new roscore, set up the environment for the ROS workspace, navigate to the ESIM package directory, run its node that creates the BAG file, launch the DVS renderer, play the BAG file and open a GUI tool to visualize the events.

In [3]:
def kill_existing_roscore():
    FNULL = open(os.devnull, 'w')
    result = subprocess.call("pgrep -f rosmaster", shell=True, stdout=FNULL)
    if result == 0:
        print("Killing existing roscore(s)...")
        subprocess.call("pkill -f rosmaster", shell=True)
        time.sleep(2)
        
def start_roscore():
    print("Starting a new roscore...")
    
    roscore_proc = subprocess.Popen("roscore", shell=True)
    time.sleep(5)  
    return roscore_proc

def run_dvs_renderer(sim_ws_setup):
    print("Launching dvs_renderer...")
    
    cmd = """
    source {} && \
    rosrun dvs_renderer dvs_renderer events:=/cam0/events
    """.format(sim_ws_setup)
    
    renderer_proc = subprocess.Popen(cmd, shell=True, executable="/bin/bash")
    return renderer_proc

def start_esim_node(sim_ws_setup, frames_path, bags_path):
    print("Launching esim_node...")
    
    cmd = """
    source {} && \
    roscd esim_ros && \
    rosrun esim_ros esim_node \
        --data_source=2 \
        --path_to_data_folder={} \
        --path_to_output_bag={}  \
        --ros_publisher_frame_rate=60 \
        --exposure_time_ms=10.0 \
        --use_log_image=1 \
        --log_eps=0.1 \
        --contrast_threshold_pos=0.15 \
        --contrast_threshold_neg=0.15 \
    """.format(sim_ws_setup, os.path.abspath(frames_path), os.path.abspath(bags_path))
    esim_proc = subprocess.Popen(cmd, shell=True, executable="/bin/bash", stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    
    renderer_proc = None
    for line in iter(esim_proc.stdout.readline, b''):
        line = line.strip()
        if "Finished writing to bag" in line:
            print("Finished writing bag file.")
            renderer_proc = run_dvs_renderer(sim_ws_setup)
            break
            
    return esim_proc, renderer_proc

def play_bag(bag_path):
    print("Reproducing the events...")
    
    cmd = """
    rosbag reindex /home/asia/out.bag && / 
    rosbag play {} -l""".format(os.path.abspath(bag_path))
    
    rosbag_proc = subprocess.Popen(cmd, shell=True)
    return rosbag_proc

def view_events():
    cmd = "rqt_image_view /dvs_rendering"
    
    viewer_proc = subprocess.Popen(cmd, shell=True)
    return viewer_proc

### Watch the events obtained from a single picture

The script below processes a single picture (defined using the frame path). It checks if the BAG file exists and, if needed, creates it before obtaining and reproducing the events.

In [13]:
sim_ws_setup = os.path.expanduser("~/sim_ws/devel/setup.bash")
sequence_root = os.path.expanduser("~/Sequences")
bag_root = os.path.expanduser("~/Bags")

specie = "Aegypius_monachus" # choose a specie from the directory: 
# Ciconia_ciconia, Ciconia_nigra, Aegypius_monachus, Gyps_fulvus, Milvus_milvus, 
# Milvus_migrans, Neophron_percnopterus, Falco_peregrinus, Aquila_chrysaetos, Aquila_adalberti
bird_number = 1 # choose a number between 1 and 400
sequence = "{}_{}".format(specie, bird_number)

sequence_path = os.path.join(sequence_root, specie, sequence)
if os.path.isdir(sequence_path):
    frames_path = os.path.join(sequence_path, "cam0", "frames")
    bag_path = os.path.join(bag_root, specie, sequence)
    out_path = os.path.join(bag_path, "out.bag")

    print("\nProcessing sequence: {}".format(sequence))
    print("Frames path: {}".format(frames_path))
    print("Bag path: {}\n".format(out_path))
    
    kill_existing_roscore()
    roscore_proc = start_roscore()

    if os.path.exists(out_path):
        print("The bag already exists.".format(sequence))
        
        esim_proc = None
        renderer_proc = run_dvs_renderer(sim_ws_setup)
        
    else:
        if not os.path.exists(bag_path):
            os.makedirs(bag_path)

        esim_proc, renderer_proc = start_esim_node(sim_ws_setup, frames_path, out_path)
        
    rosbag_proc = play_bag(out_path)
    viewer_proc = view_events()
else:
    print("\nThe choosen path doesn't exist: {}".format(sequence_path))


Processing sequence: Aegypius_monachus_1
Frames path: /home/asia/Sequences/Aegypius_monachus/Aegypius_monachus_1/cam0/frames
Bag path: /home/asia/Bags/Aegypius_monachus/Aegypius_monachus_1/out.bag

Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...
Finished writing bag file.
Launching dvs_renderer...
Reproducing the events...


### Save the events obtained from multiple pictures

The script below converts the whole dataset from frames to BAG files, without reproducing any of the events.

In [ ]:
sim_ws_setup = os.path.expanduser("~/sim_ws/devel/setup.bash")
sequence_root = os.path.expanduser("~/Sequences")
bag_root = os.path.expanduser("~/Bags")

found_sequences = False 
processed_any = False 
        
for specie in sorted(os.listdir(sequence_root)):
    specie_path = os.path.join(sequence_root, specie)
    if not os.path.isdir(specie_path):
        continue

    for sequence in sorted(os.listdir(specie_path)):
        sequence_path = os.path.join(specie_path, sequence)
        if not os.path.isdir(sequence_path):
            continue
        
        found_sequences = True

        frames_path = os.path.join(sequence_path, "cam0", "frames")
        bag_path = os.path.join(bag_root, specie, sequence)
        out_path = os.path.join(bag_path, "out.bag")

        if os.path.exists(bag_path):
            print("\nSkipping: {}. The bag already exists.".format(sequence))
            continue

        print("\nProcessing: {}".format(sequence))
        print("Frames path: {}".format(frames_path))
        print("Bag path: {}".format(out_path))

        if not os.path.exists(bag_path):
            os.makedirs(bag_path)

        kill_existing_roscore()
        roscore_proc = start_roscore()
        esim_proc, renderer_proc = start_esim_node(sim_ws_setup, frames_path, out_path)

        processed_any = True
        
if not found_sequences:
    print("\nThere are no available sequences.")
elif not processed_any:
    print("\nAll sequences have already been processed.")
else:
    print("\nAll sequences have been processed.")

## Putting all together

The code below creates the frames and the bag for each image, without splitting the two parts of the process. Please remember to execute the cells with the libraries (at the start of this file) and the ones with the needed functions (which are the first one in the section "Create the frames" and the first one in the section "Obtain the events").

In [4]:
image_root = os.path.expanduser("~/Images")
sequence_root = os.path.expanduser("~/Sequences")
label_root = os.path.expanduser("~/Labels")
bag_root = os.path.expanduser("~/Bags")
sim_ws_setup = os.path.expanduser("~/sim_ws/devel/setup.bash")

num_frames = 100
zoom_min = 1.0
zoom_max = 5.0
square_size = 512

if not os.path.isdir(sequence_root):
    os.makedirs(sequence_root)

if not os.path.isdir(bag_root):
    os.makedirs(bag_root)

for specie in sorted(os.listdir(image_root)):
    specie_path = os.path.join(image_root, specie)
    if not os.path.isdir(specie_path):
        continue

    for image_name in sorted(os.listdir(specie_path)):
        if not image_name.lower().endswith(".png"):
            continue

        image_path = os.path.join(specie_path, image_name)
        img = cv2.imread(image_path)
        if img is None:
            continue

        base_name = os.path.splitext(image_name)[0]
        sequence = "{}_{}".format(specie, base_name)

        sequence_path = os.path.join(sequence_root, specie, sequence)
        frames_path = os.path.join(sequence_path, "cam0", "frames")
        label_path = os.path.join(label_root, specie, "{}.{}".format(base_name, "txt"))
        bag_path = os.path.join(bag_root, specie, sequence)
        out_path = os.path.join(bag_path, "out.bag")
        
        if os.path.exists(out_path):
            print("\nSkipping already processed sequence: {}".format(sequence))
            continue
            
        print("\nProcessing: {}".format(sequence))
        
        if not os.path.exists(label_path):
            print("Label file not found for image: {}".format(image_path))
            continue
        
        if not os.path.exists(frames_path):
            os.makedirs(frames_path)
        print("Frames path: {}".format(frames_path))
        
        print("Label path: {}".format(label_path))

        if not os.path.exists(bag_path):
            os.makedirs(bag_path)            
        print("Bag path: {}".format(out_path))
        
        h, w = img.shape[:2]
        bbox = read_yolo(label_path, w, h)
        cropped_img = crop_square(img, bbox, square_size)
        
        for i in range(num_frames):
            zoom = zoom_min + (zoom_max - zoom_min) * i / (num_frames - 1)
            frame = zoom_image(cropped_img, zoom)
            frame_name = "frame_{}.png".format(str(i).zfill(3))
            cv2.imwrite(os.path.join(frames_path, frame_name), frame)

        cmd = "python /home/asia/sim_ws/src/rpg_esim/event_camera_simulator/esim_ros/scripts/generate_stamps_file.py -i {} -r 50.0".format(os.path.abspath(frames_path))
        subprocess.call(cmd, shell=True)

        kill_existing_roscore()
        roscore_proc = start_roscore()
        esim_proc, roscore_proc = start_esim_node(sim_ws_setup, frames_path, out_path)

        if os.path.exists(sequence_path):
            shutil.rmtree(sequence_path)
            print("Removed frames folder: {}".format(sequence_path))

print("\nAll images processed.")


Processing: Aquila_chrysaetos_1
Frames path: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_1/cam0/frames
Label path: /home/asia/Labels/Aquila_chrysaetos/1.txt
Bag path: /home/asia/Bags/Aquila_chrysaetos/Aquila_chrysaetos_1/out.bag
Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...
Finished writing bag file.
Launching dvs_renderer...
Removed frames folder: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_1

Processing: Aquila_chrysaetos_10
Frames path: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_10/cam0/frames
Label path: /home/asia/Labels/Aquila_chrysaetos/10.txt
Bag path: /home/asia/Bags/Aquila_chrysaetos/Aquila_chrysaetos_10/out.bag
Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...
Finished writing bag file.
Launching dvs_renderer...
Removed frames folder: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_10

Processing: Aquila_chrysaetos_100
Frames path: /home/asia/Sequences/Aquila_chr

Starting a new roscore...
Launching esim_node...
Finished writing bag file.
Launching dvs_renderer...
Removed frames folder: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_114

Processing: Aquila_chrysaetos_115
Frames path: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_115/cam0/frames
Label path: /home/asia/Labels/Aquila_chrysaetos/115.txt
Bag path: /home/asia/Bags/Aquila_chrysaetos/Aquila_chrysaetos_115/out.bag
Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...
Finished writing bag file.
Launching dvs_renderer...
Removed frames folder: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_115

Processing: Aquila_chrysaetos_116
Frames path: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_116/cam0/frames
Label path: /home/asia/Labels/Aquila_chrysaetos/116.txt
Bag path: /home/asia/Bags/Aquila_chrysaetos/Aquila_chrysaetos_116/out.bag
Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...
Finished writin

Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...
Finished writing bag file.
Launching dvs_renderer...
Removed frames folder: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_130

Processing: Aquila_chrysaetos_131
Frames path: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_131/cam0/frames
Label path: /home/asia/Labels/Aquila_chrysaetos/131.txt
Bag path: /home/asia/Bags/Aquila_chrysaetos/Aquila_chrysaetos_131/out.bag
Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...
Finished writing bag file.
Launching dvs_renderer...
Removed frames folder: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_131

Processing: Aquila_chrysaetos_132
Frames path: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_132/cam0/frames
Label path: /home/asia/Labels/Aquila_chrysaetos/132.txt
Bag path: /home/asia/Bags/Aquila_chrysaetos/Aquila_chrysaetos_132/out.bag
Killing existing roscore(s)...
Starting a new roscore...
Launchi

Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...
Finished writing bag file.
Launching dvs_renderer...
Removed frames folder: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_147

Processing: Aquila_chrysaetos_148
Frames path: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_148/cam0/frames
Label path: /home/asia/Labels/Aquila_chrysaetos/148.txt
Bag path: /home/asia/Bags/Aquila_chrysaetos/Aquila_chrysaetos_148/out.bag
Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...
Finished writing bag file.
Launching dvs_renderer...
Removed frames folder: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_148

Processing: Aquila_chrysaetos_149
Frames path: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_149/cam0/frames
Label path: /home/asia/Labels/Aquila_chrysaetos/149.txt
Bag path: /home/asia/Bags/Aquila_chrysaetos/Aquila_chrysaetos_149/out.bag
Killing existing roscore(s)...
Starting a new roscore...
Launchi

Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...
Finished writing bag file.
Launching dvs_renderer...
Removed frames folder: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_163

Processing: Aquila_chrysaetos_164
Frames path: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_164/cam0/frames
Label path: /home/asia/Labels/Aquila_chrysaetos/164.txt
Bag path: /home/asia/Bags/Aquila_chrysaetos/Aquila_chrysaetos_164/out.bag
Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...
Finished writing bag file.
Launching dvs_renderer...
Removed frames folder: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_164

Processing: Aquila_chrysaetos_165
Frames path: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_165/cam0/frames
Label path: /home/asia/Labels/Aquila_chrysaetos/165.txt
Bag path: /home/asia/Bags/Aquila_chrysaetos/Aquila_chrysaetos_165/out.bag
Killing existing roscore(s)...
Starting a new roscore...
Launchi

...

Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...
Finished writing bag file.
Launching dvs_renderer...
Removed frames folder: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_390

Processing: Aquila_chrysaetos_391
Frames path: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_391/cam0/frames
Label path: /home/asia/Labels/Aquila_chrysaetos/391.txt
Bag path: /home/asia/Bags/Aquila_chrysaetos/Aquila_chrysaetos_391/out.bag
Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...
Finished writing bag file.
Launching dvs_renderer...
Removed frames folder: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_391

Processing: Aquila_chrysaetos_392
Frames path: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_392/cam0/frames
Label path: /home/asia/Labels/Aquila_chrysaetos/392.txt
Bag path: /home/asia/Bags/Aquila_chrysaetos/Aquila_chrysaetos_392/out.bag
Killing existing roscore(s)...
Starting a new roscore...
Launchi

Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...
Finished writing bag file.
Launching dvs_renderer...
Removed frames folder: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_46

Processing: Aquila_chrysaetos_47
Frames path: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_47/cam0/frames
Label path: /home/asia/Labels/Aquila_chrysaetos/47.txt
Bag path: /home/asia/Bags/Aquila_chrysaetos/Aquila_chrysaetos_47/out.bag
Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...
Finished writing bag file.
Launching dvs_renderer...
Removed frames folder: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_47

Processing: Aquila_chrysaetos_48
Frames path: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_48/cam0/frames
Label path: /home/asia/Labels/Aquila_chrysaetos/48.txt
Bag path: /home/asia/Bags/Aquila_chrysaetos/Aquila_chrysaetos_48/out.bag
Killing existing roscore(s)...
Starting a new roscore...
Launching esim_no

Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...
Finished writing bag file.
Launching dvs_renderer...
Removed frames folder: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_62

Processing: Aquila_chrysaetos_63
Frames path: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_63/cam0/frames
Label path: /home/asia/Labels/Aquila_chrysaetos/63.txt
Bag path: /home/asia/Bags/Aquila_chrysaetos/Aquila_chrysaetos_63/out.bag
Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...
Finished writing bag file.
Launching dvs_renderer...
Removed frames folder: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_63

Processing: Aquila_chrysaetos_64
Frames path: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_64/cam0/frames
Label path: /home/asia/Labels/Aquila_chrysaetos/64.txt
Bag path: /home/asia/Bags/Aquila_chrysaetos/Aquila_chrysaetos_64/out.bag
Killing existing roscore(s)...
Starting a new roscore...
Launching esim_no

Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...
Finished writing bag file.
Launching dvs_renderer...
Removed frames folder: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_79

Processing: Aquila_chrysaetos_8
Frames path: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_8/cam0/frames
Label path: /home/asia/Labels/Aquila_chrysaetos/8.txt
Bag path: /home/asia/Bags/Aquila_chrysaetos/Aquila_chrysaetos_8/out.bag
Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...
Finished writing bag file.
Launching dvs_renderer...
Removed frames folder: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_8

Processing: Aquila_chrysaetos_80
Frames path: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_80/cam0/frames
Label path: /home/asia/Labels/Aquila_chrysaetos/80.txt
Bag path: /home/asia/Bags/Aquila_chrysaetos/Aquila_chrysaetos_80/out.bag
Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...

Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...
Finished writing bag file.
Launching dvs_renderer...
Removed frames folder: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_95

Processing: Aquila_chrysaetos_96
Frames path: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_96/cam0/frames
Label path: /home/asia/Labels/Aquila_chrysaetos/96.txt
Bag path: /home/asia/Bags/Aquila_chrysaetos/Aquila_chrysaetos_96/out.bag
Killing existing roscore(s)...
Starting a new roscore...
Launching esim_node...
Finished writing bag file.
Launching dvs_renderer...
Removed frames folder: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_96

Processing: Aquila_chrysaetos_97
Frames path: /home/asia/Sequences/Aquila_chrysaetos/Aquila_chrysaetos_97/cam0/frames
Label path: /home/asia/Labels/Aquila_chrysaetos/97.txt
Bag path: /home/asia/Bags/Aquila_chrysaetos/Aquila_chrysaetos_97/out.bag
Killing existing roscore(s)...
Starting a new roscore...
Launching esim_no